In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense, BatchNormalization
from keras import applications
from datetime import datetime as dt
from keras import regularizers as reg
from keras.optimizers import RMSprop

import warnings
warnings.filterwarnings("ignore")

In [2]:
global_start=dt.now()

#Dimensions of our flicker images is 256 X 256
img_width, img_height = 256, 256

#Declaration of parameters needed for training and validation
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 3600 #1200 training samples for each class
nb_validation_samples = 1200 #400 training samples for each class
epochs = 50
batch_size = 16


In [7]:
global_start=dt.now()

#Dimensions of our flicker images is 256 X 256
img_width, img_height = 256, 256

#Declaration of parameters needed for training and validation
train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 3600 #1200 training samples for each class
nb_validation_samples = 1200 #400 training samples for each class
epochs = 40
batch_size = 16

#Get the bottleneck features by  Weights.T * Xi
def save_bottlebeck_features():
    datagen = ImageDataGenerator(rescale=1./255)

    #Load the pre trained VGG16 model from Keras, we will initialize only the convolution layers and ignore the top layers.
    model = applications.VGG16(include_top=False, weights='imagenet')

    generator = datagen.flow_from_directory(train_data_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=batch_size,
                                            class_mode=None, #class_mode=None means the generator won't load the class labels.
                                            shuffle=False) #We won't shuffle the data, because we want the class labels to stay in order.
    bottleneck_features_train = model.predict_generator(generator, nb_train_samples // batch_size)
    np.save('weights/bottleneck_features_train.npy',bottleneck_features_train) #bottleneck_features_train is a numpy array

    generator = datagen.flow_from_directory(validation_data_dir,
                                            target_size=(img_width, img_height),
                                            batch_size=batch_size,
                                            class_mode=None,
                                            shuffle=False)
    bottleneck_features_validation = model.predict_generator(generator, nb_validation_samples // batch_size)
    np.save('weights/bottleneck_features_validation.npy',bottleneck_features_validation)
    print("Got the bottleneck features in time: ",dt.now()-global_start)
    
save_bottlebeck_features()

Found 3600 images belonging to 3 classes.
Found 1200 images belonging to 3 classes.
Got the bottleneck features in time:  0:45:59.643704


In [11]:
#After we get the bottleneck features, we will build the top fully connected layers on top of the bottlneck features. Let's build the top layers.
def train_top_model():
    global_start=dt.now()

    train_data = np.load('weights/bottleneck_features_train.npy')
    train_labels = np.array([0] * (nb_train_samples // 3) + [1] * (nb_train_samples // 3) + [2] * (nb_train_samples // 3)) #Equivalent to: np.array([0]*1200 + [1]*1200 + [2]*1200)

    validation_data = np.load('weights/bottleneck_features_validation.npy')
    validation_labels = np.array([0] * (nb_validation_samples // 3) + [1] * (nb_validation_samples // 3) + [2] * (nb_validation_samples // 3))

    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:])) #Ignore the first index. It contains ID

    model.add(Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001))) #Best weight initializer for relu is he_normal
    model.add(BatchNormalization()) #Add a BatchNormalization layer to control internel covariance shift
    model.add(Dropout(rate=0.5)) #Using droput for regularization

    model.add(Dense(256, activation='relu',kernel_initializer='he_normal',kernel_regularizer=reg.l1_l2(l1=0.001, l2=0.001)))
    model.add(BatchNormalization()) #Add a BatchNormalization layer to control internel covariance shift
    model.add(Dropout(rate=0.5))

    model.add(Dense(3, activation='softmax',kernel_initializer='glorot_uniform')) #Because we have 3 classes. Remember, softmax is to multi-class, what sigmoid (log reg) is to binary

    optim=RMSprop(lr=0.0001, epsilon=1e-8, decay=1e-6)
    model.compile(loss='sparse_categorical_crossentropy',optimizer=optim,metrics=['accuracy'])
    model.summary()

    model.fit(x=train_data,
              y=train_labels,
              epochs=epochs,
              validation_data=(validation_data, validation_labels))    
    model.save_weights('weights/bottleneck_feats_multi_weights.h5') 
    model.save('weights/bottleneck_feats_multi_model.h5')
    print("The top layer trained in time: ",dt.now()-global_start)
    
    return model


model=train_top_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_8 (Flatten)          (None, 32768)             0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               8388864   
_________________________________________________________________
batch_normalization_15 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_15 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 256)               65792     
_________________________________________________________________
batch_normalization_16 (Batc (None, 256)               1024      
_________________________________________________________________
dropout_16 (Dropout)         (None, 256)               0         
__________

In [5]:
#Load the top model
from keras.models import load_model
model = load_model('weights/bottleneck_feats_multi_model.h5')
model.predict()

In [3]:
#This function is used to plot/update the train and test loss after each epoch.
import matplotlib.pyplot as plt
nb_epoch=40
def plt_dynamic_loss(x, vy, ty, ax, colors=['b']):
    ax.plot(x, vy, 'b', label="Validation Loss")
    ax.plot(x, ty, 'r', label="Train Loss")
    plt.legend()
    plt.grid()
    fig.canvas.draw()

#Get model history
history=model.history

'''#Plot the train and test loss vs number of epochs
score = model.evaluate(X_test, y_test, verbose=0) 
print('Test score (Validation Loss):', score[0]) 
print('Test accuracy (Accuracy on Unseen Data):', score[1])'''

#Plot train vs test loss
fig,ax = plt.subplots(1,1)
ax.set_xlabel('Epoch') ; ax.set_ylabel('Categorical Crossentropy Loss')

#List of epoch numbers
x = list(range(1,nb_epoch+1))

#Display the loss
val_loss = history.history['val_loss'] #Validation Loss
loss = history.history['loss'] #Training Loss
plt_dynamic_loss(x, val_loss, loss, ax)

AttributeError: 'Sequential' object has no attribute 'history'

In [13]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from keras.models import Sequential, load_model
import time

start = time.time()

#Load the pre-trained models
model = load_model('   bottleneck_feats_multi_model.h5')
model.load_weights('bottleneck_feats_multi_weights.h5')

#Define image parameters. This should be same as the dimensions of the input image
img_width, img_height = 256, 256

#Make predictions using this function. Call this function with the file name
def predict(file):
    img = load_img(file, target_size=(img_width,img_height))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    array = model.predict(img)
    result = array[0]
    #print(result)
    answer = np.argmax(result)
    if answer == 1:
        print("Predicted: Tiger.")
    elif answer == 0:
        print("Predicted: Horse")
    elif answer == 2:
        print("Predicted: Monkey")

    return answer

#Walk the directory for every image
for i, ret in enumerate(os.walk(test_path)):
    for i, filename in enumerate(ret[2]):
        if (filename.startswith(".")):
            continue
    
    print(ret[0] + '/' + filename)
    result = predict(ret[0] + '/' + filename)
    print(" ")

#Calculate execution time
end = time.time()
dur = end-start

if dur<60:
    print("Execution Time:",dur,"seconds")
elif dur>60 and dur<3600:
    dur=dur/60
    print("Execution Time:",dur,"minutes")
else:
    dur=dur/(60*60)
print("Execution Time:",dur,"hours")

Execution Time: 5.367101192474365 seconds
Execution Time: 5.367101192474365 hours
